# Introduction

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a 
dataframe. This allows each column to have multiple keys associated with it--one key for each level. 

This makes it easy to associate multiple identifiers with a single column. To show what this looks like we'll look at phosphoproteomics data in the endometrial cancer dataset.

In [1]:
import cptac
cptac.download(dataset="endometrial", version="latest")
en = cptac.Endometrial()
phospho = en.get_phosphoproteomics()
phospho.head()

Name        AAAS                  AACS  AAED1 AAGAB          AAK1         \
Site        S495    S541   Y485   S618    S12  S310   S311    S14    S18   
Sample_ID                                                                  
S001         NaN     NaN    NaN -0.881 -1.810   NaN    NaN    NaN -0.242   
S002         NaN     NaN    NaN    NaN  0.084   NaN    NaN -1.110 -0.383   
S003      -0.202     NaN    NaN    NaN -1.880   NaN    NaN    NaN  0.382   
S005      -0.002     NaN -0.407    NaN    NaN   NaN    NaN    NaN    NaN   
S006       0.556 -0.0461    NaN    NaN  0.941   NaN  0.429  0.362  0.697   

Name               ...     ZZZ3                                            \
Site          S20  ...     S397 S411 S420   S424     S426 S468   S89 T415   
Sample_ID          ...                                                      
S001      -0.2420  ...  0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
S002      -1.0900  ... -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
S003      -0.0416  ...      NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
S005      -0.5550  ...  0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
S006      -0.0529  ... -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                       
Site         T418    Y399  
Sample_ID                  
S001          NaN     NaN  
S002       0.1605 -0.0635  
S003          NaN     NaN  
S005          NaN  0.2980  
S006          NaN -0.2900  

[5 rows x 73212 columns]

The first level of keys is the name of the gene associated with the column of data. The second level is the site of acetylation. Each column has its own Name and Site associated with it. 
Normally, as above, keys that repeat for adjacent columns are not displayed, to make it easier to read the dataframe. For example, in the table above the first 7 columns are all for the A2M gene, but the label is only shown for the first column. 

# Join functions with multiindices 
The join functions have been written to handle multiindices. For more information on the join functions, see joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [2]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

Name      A1BG_CNV A1BG-AS1_CNV A1CF_CNV A2M_CNV A2M-AS1_CNV A2ML1_CNV  \
Site           NaN          NaN      NaN     NaN         NaN       NaN   
Sample_ID                                                                
S001          0.00         0.00    -0.01    0.00        0.00      0.00   
S002          0.01         0.01     0.00    0.00        0.00      0.00   
S003          0.01         0.01     0.00    0.00        0.00      0.00   
S005         -0.04        -0.04     0.36   -0.05       -0.05     -0.05   
S006          0.85         0.85     0.16   -0.41       -0.41     -0.41   

Name      A2MP1_CNV A3GALT2_CNV A4GALT_CNV A4GNT_CNV  ...  \
Site            NaN         NaN        NaN       NaN  ...   
Sample_ID                                             ...   
S001           0.00        0.00       0.01     -0.01  ...   
S002           0.00        0.00       0.01      0.00  ...   
S003           0.00        0.01       0.00      0.00  ...   
S005          -0.05       -0.04      -0.04     -0.05  ...   
S006          -0.41       -0.12       0.15      0.19  ...   

Name      ZZZ3_phosphoproteomics                                            \
Site                        S397 S411 S420   S424     S426 S468   S89 T415   
Sample_ID                                                                    
S001                     0.18400  NaN  NaN    NaN -0.20500  NaN   NaN  NaN   
S002                    -0.17100  NaN  NaN -0.393 -0.17100  NaN  0.29  NaN   
S003                         NaN  NaN  NaN    NaN      NaN  NaN   NaN  NaN   
S005                     0.13970  NaN  NaN    NaN -0.55900  NaN   NaN  NaN   
S006                    -0.15875  NaN  NaN  0.196  0.06175  NaN   NaN  NaN   

Name                       
Site         T418    Y399  
Sample_ID                  
S001          NaN     NaN  
S002       0.1605 -0.0635  
S003          NaN     NaN  
S005          NaN  0.2980  
S006          NaN -0.2900  

[5 rows x 101269 columns]

Since the CNV dataframe doesn't have the level "Site", it is filled in with NANs, so that it can be joined to the phosphoproteomics dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the acetylproteomics for A2M at site K1176, we'd do the following:

In [3]:
acetyl = en.get_acetylproteomics()
all_levels_selection = acetyl["A2M"]["K1176"]
all_levels_selection.head(10)

Sample_ID
S001    1.080
S002    0.477
S003      NaN
S005   -0.608
S006    1.630
S007   -0.210
S008   -0.175
S009   -0.223
S010   -1.120
S011      NaN
Name: K1176, dtype: float64

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [4]:
gene1_filter = acetyl.columns.get_level_values("Name").str.startswith("AA") # Select all columns where the gene starts with "AA". This will grab every column where the key "Name" starts with AA
gene1_data = acetyl.loc[:, gene1_filter]
gene1_data.head()

Name      AACS  AAGAB     AAK1 AARS AASDH AASS               AATF    
Site      K391   K290     K201 K338  K765 K138 K539    K93   K380 K74
Sample_ID                                                            
S001       NaN  0.461      NaN  NaN   NaN  NaN  NaN    NaN  1.300 NaN
S002       NaN  1.770      NaN  NaN   NaN  NaN  NaN    NaN  0.652 NaN
S003       NaN -0.815 -0.00573  NaN   NaN  NaN  NaN -0.305  0.398 NaN
S005       NaN    NaN      NaN  NaN   NaN  NaN  NaN    NaN    NaN NaN
S006       NaN  0.205      NaN  NaN   NaN  NaN  NaN    NaN  0.244 NaN

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get all sites from all genes where tyrosine is phosphorylated:

In [5]:
y_site_filter = phospho.columns.get_level_values("Site").str.contains("Y") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = phospho.loc[:, y_site_filter] # Select the columns
y_sites.head()

Name        AAAS    AAK1   ABCC1   ABI1 ABI2 ABI3   ABL1   ABL2 ABLIM1         \
Site        Y485    Y687    Y920   Y213 Y213 Y361   Y204    Y80   Y357   Y439   
Sample_ID                                                                       
S001         NaN     NaN  0.4340 -0.591  NaN  NaN -0.759    NaN    NaN  0.312   
S002         NaN     NaN -0.5110    NaN  NaN  NaN -1.210  0.321    NaN    NaN   
S003         NaN     NaN     NaN    NaN  NaN  NaN    NaN    NaN    NaN    NaN   
S005      -0.407     NaN  1.2680    NaN  NaN  NaN -0.173    NaN    NaN    NaN   
S006         NaN -0.4315 -0.0685    NaN  NaN  NaN -0.965  0.657    NaN    NaN   

Name       ... ZNF799  ZNF839 ZNF860  ZNRF2 ZRANB2         ZRSR2    ZYX ZZEF1  \
Site       ...   Y430    Y283    Y49   Y111   Y114   Y124   Y347   Y172 Y1795   
Sample_ID  ...                                                                  
S001       ...    NaN     NaN    NaN    NaN    NaN  0.181  0.788 -0.378   NaN   
S002       ...    NaN  0.0684    NaN    NaN    NaN -0.929    NaN -1.230   NaN   
S003       ...    NaN     NaN    NaN    NaN    NaN -0.283  0.675  0.700   NaN   
S005       ...    NaN     NaN    NaN    NaN    NaN -0.123 -0.026 -1.020   NaN   
S006       ...    NaN     NaN    NaN -0.058    NaN  0.109    NaN -0.636   NaN   

Name         ZZZ3  
Site         Y399  
Sample_ID          
S001          NaN  
S002      -0.0635  
S003          NaN  
S005       0.2980  
S006      -0.2900  

[5 rows x 2397 columns]

# How to use DataSet.reduce_multiindex()
To make it easier to work with multi-level indices, we have provide the DataSet.reduce_multiindex function. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [6]:
en.reduce_multiindex(df=phospho, levels_to_drop="Site").head()

Name,AAAS,AAAS,AAAS,AACS,AAED1,AAGAB,AAGAB,AAK1,AAK1,AAK1,...,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3,ZZZ3
Sample_ID,,,,,,,,,,,,,,,,,,,,,
S001,NaN,NaN,NaN,-0.881,-1.810,NaN,NaN,NaN,-0.242,-0.2420,...,0.18400,NaN,NaN,NaN,-0.20500,NaN,NaN,NaN,NaN,NaN
S002,NaN,NaN,NaN,NaN,0.084,NaN,NaN,-1.110,-0.383,-1.0900,...,-0.17100,NaN,NaN,-0.393,-0.17100,NaN,0.29,NaN,0.1605,-0.0635
S003,-0.202,NaN,NaN,NaN,-1.880,NaN,NaN,NaN,0.382,-0.0416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
S005,-0.002,NaN,-0.407,NaN,NaN,NaN,NaN,NaN,NaN,-0.5550,...,0.13970,NaN,NaN,NaN,-0.55900,NaN,NaN,NaN,NaN,0.2980
S006,0.556,-0.0461,NaN,NaN,0.941,NaN,0.429,0.362,0.697,-0.0529,...,-0.15875,NaN,NaN,0.196,0.06175,NaN,NaN,NaN,NaN,-0.2900


### Dropping single or multiple at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 